In [102]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import animation
from skimage import transform
from tqdm.auto import tqdm
from IPython.display import HTML
import ipywidgets as widgets

In [119]:
# This normalizes intensities for across a puff montage and returns
# a numpy array with dims (x,y,num_frames) for display as if a tif
def puff_to_frames(puff):
    # get properties for display
    num_frames = len(np.unique(puff['frame']))
    dim_x = len(np.unique(puff.loc[puff['frame'] == puff.loc[0,'frame'],'x']))
    dim_y = len(np.unique(puff.loc[puff['frame'] == puff.loc[0,'frame'],'y']))
    scale_factor = 2
    
    # iterate through frames and convert to dim_x x dim_y matrix
    puff_matrix = np.zeros((dim_x*scale_factor,
                            dim_y*scale_factor,
                            num_frames),
                           dtype=np.uint16)
    for frame in np.unique(puff['frame']):
        relframe = frame - np.min(puff['frame'])
        cur_frame = puff[puff['frame']==frame].pivot_table(values=['intensity'],columns=['x'],index=['y']).values
        cur_frame = transform.rescale(cur_frame, 2,
                                      multichannel=False,
                                      preserve_range=True)
        puff_matrix[:,:,int(relframe)] = cur_frame
        
    return puff_matrix

def generate_widget(puff_frames, cell_name, puff_num):
    obj_layout = widgets.Layout(width='200px')
    out = widgets.Output()
    cluster = widgets.Dropdown(options = np.arange(1,11).tolist(),
                               value = 1,
                               layout = widgets.Layout(width='150px'),
                               description = 'Cluster:')
    label = widgets.Label(value='Cell ' + cell_name + ': Puff ' + puff_num,
                         layout=obj_layout)
    play = widgets.Play(interval=200, value=0,
                        min=0, max=puff_frames.shape[2]-1, 
                        step=1, description="Press play",
                        disabled=False)
    slider = widgets.IntSlider(min=0, max=puff_frames.shape[2]-1,
                               layout=obj_layout)
    widgets.jslink((play, 'value'), (slider, 'value'))
    ui = widgets.VBox([label, play, slider, out, cluster], 
                      layout=widgets.Layout(width='210',align_items='stretch'))

    vmin = np.min(puff_frames.flatten())
    vmax = np.max(puff_frames.flatten())
    
    def paint_puff(frame):
        with out:
            out.clear_output()
            fig, ax = plt.subplots(figsize=(3,3))
            ax.axis('off')
            plt.imshow(puff_frames[:,:,frame],
                  vmin=vmin, vmax=vmax, cmap='gray')

    int_out = widgets.interactive_output(paint_puff,{'frame': slider})
    return widgets.VBox([ui, int_out])

In [141]:
def score_some_clusters(scorer, cell_name):
    puffs = pd.read_csv(cell_name + '.tif_puff_intensities.csv',index_col=0)

    np.random.seed(237)
    puff_ids = np.random.choice(np.unique(puffs['particle']),20,replace=False)
    widget_list = []
    for puff_id in tqdm(puff_ids):
        frames = puff_to_frames(puffs.loc[puffs['particle']==puff_id, :])
        widget = generate_widget(frames, cell_name, str(puff_id))
        widget_list.append(widget)
        
    ids_with_clusters = pd.DataFrame(columns=['particle','cluster'])
    ids_with_clusters['particle'] = puff_ids

    puff_boxes = widgets.HBox(widget_list,layout=widgets.Layout(flex_flow = 'row wrap'))
    butt = widgets.Button(description = 'Save Clusters',
                          disabled=False,
                          button_style='')
    display(widgets.VBox([puff_boxes,butt]))

    def save_clusters(b):
        for idx, particle in enumerate(puff_ids):
            cluster_id = widget_list[idx].children[0].children[4].value
            ids_with_clusters.loc[idx,'cluster'] = cluster_id
            ids_with_clusters.to_csv(cell_name + '_clusters_' + scorer + '.csv')

    butt.on_click(save_clusters)

### Who are you??

In [155]:
scorer = 'Caroline'

# 19CC75ZU Movie

In [156]:
cell_name = '19CC75ZU'
score_some_clusters(scorer, cell_name)

# 0IYVHRNA Movie

In [157]:
cell_name = '0IYVHRNA'
score_some_clusters(scorer, cell_name)

# 2TZWB6CN Movie

In [158]:
cell_name = '2TZWB6CN'
score_some_clusters(scorer, cell_name)

# 3W70AV4V Movie

In [159]:
cell_name = '3W70AV4V'
score_some_clusters(scorer, cell_name)